In [1]:
# implementation of GPT from scratch (but charcter level) and train it on tiny shaksphere

# 2017, Attention is all you need 
# GPT -> Generatively Pretrained Transformer 

In [2]:
with open("data/input.txt", 'r', encoding='utf-8')as f:
    text = f.read()

In [3]:
print(f"no of charcters = {len(text)}")
print(f"no of line= {len(text.splitlines())}")


no of charcters = 1115394
no of line= 40000


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# get all characters 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"vocab_size: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size: 65


In [6]:
# create a mapping from chars to int and vise-versa
# character level tokeniser 
# long sequences 
stoi = {s:i for i,s in enumerate(chars)}
itos = {v:k for k,v in stoi.items()}
# How lambda works--> some_fucntion_object = lambda input:output_(input)
encode = lambda  s:[stoi[c] for c in s] #encoder: take a string, output a list of integer
decode = lambda  l: ''.join([itos[i] for i in l]) # decode: takes a list of indexes and output a string

In [7]:
# test the encoder and decoder 
print(encode("hello world!"))
print(decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]))
print(decode(encode("hello world!"))) # if it prints "hello world!" it works !

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello world!
hello world!


In [8]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data.dtype)
print(data[:10])

torch.Size([1115394])
torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [9]:
# train and validation split 
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
# when the input is {context} the target is {target}
block_size = 8 # time dimention
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    context_itos = [itos[i.item()] for i in context]
    target = y[t]
    target_itos = itos[target.item()]
    print(f"when the context is {context} ->target is {target}")
    print(f"when the context is {context_itos} ->target is '{target_itos}'")
    print("---------")

when the context is tensor([18]) ->target is 47
when the context is ['F'] ->target is 'i'
---------
when the context is tensor([18, 47]) ->target is 56
when the context is ['F', 'i'] ->target is 'r'
---------
when the context is tensor([18, 47, 56]) ->target is 57
when the context is ['F', 'i', 'r'] ->target is 's'
---------
when the context is tensor([18, 47, 56, 57]) ->target is 58
when the context is ['F', 'i', 'r', 's'] ->target is 't'
---------
when the context is tensor([18, 47, 56, 57, 58]) ->target is 1
when the context is ['F', 'i', 'r', 's', 't'] ->target is ' '
---------
when the context is tensor([18, 47, 56, 57, 58,  1]) ->target is 15
when the context is ['F', 'i', 'r', 's', 't', ' '] ->target is 'C'
---------
when the context is tensor([18, 47, 56, 57, 58,  1, 15]) ->target is 47
when the context is ['F', 'i', 'r', 's', 't', ' ', 'C'] ->target is 'i'
---------
when the context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) ->target is 58
when the context is ['F', 'i', 'r', 

In [ ]:
# Batch dimention
torch.manual_seed(1337)

block_size = 8 # what is the length of each sequence 
batch_size = 4 # how many independent sequences we will process in parallel 

def get_batch(split):
    """
    Get a mini batch from specified data split
    input: "train" or "val"
    output: x, y 
    y is x moved to right by an index 
    x.shape -> [4,8]
    yb.shape -> [4,8]
    """
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size-1,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch("train")
print(xb)
print(xb.shape)
print(yb)
print(yb.shape)

print("---------")
# -----
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        context_itos = [itos[i.item()] for i in context]
        target = yb[b,t]
        target_itos = itos[target.item()]
        print(f"printing sequence:{b+1}, example: {t+1}")
        print(f"when the context is {context_itos} ->target is '{target_itos}'")
        print("---------")

tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
torch.Size([4, 8])
---------
printing sequence:1, example: 1
when the context is ['o'] ->target is 'u'
---------
printing sequence:1, example: 2
when the context is ['o', 'u'] ->target is ','
---------
printing sequence:1, example: 3
when the context is ['o', 'u', ','] ->target is ' '
---------
printing sequence:1, example: 4
when the context is ['o', 'u', ',', ' '] ->target is 't'
---------
printing sequence:1, example: 5
when the context is ['o', 'u', ',', ' ', 't'] ->target is 'r'
---------
printing sequence:1, example: 6
when the context is ['o', 'u', ',', ' ', 't', 'r'] ->target is 'i'
---------
printing sequence:1, example: 7
whe